In [ ]:
# Nilesh Kale (AI3005)

import pandas as pd
path="/content/drive/MyDrive/heart_2022.csv"
df=pd.read_csv(path)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Identify categorical and numerical columns
#categorical_cols = ["State", "Sex", "GeneralHealth", "LastCheckupTime", "PhysicalActivities", "RemovedTeeth"]
#numerical_cols = ["PhysicalHealthDays", "MentalHealthDays", "SleepHours"]

# Automatically identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()  # Select string-based categorical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()  # Select numerical columns

# Print results
print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)


Categorical Columns: ['State', 'Sex', 'GeneralHealth', 'LastCheckupTime', 'PhysicalActivities', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina', 'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD', 'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis', 'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty', 'DifficultyConcentrating', 'DifficultyWalking', 'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus', 'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory', 'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap', 'HighRiskLastYear', 'CovidPos']
Numerical Columns: ['PhysicalHealthDays', 'MentalHealthDays', 'SleepHours', 'HeightInMeters', 'WeightInKilograms', 'BMI']


In [ ]:
# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save encoders for later use

In [ ]:
print(df.head())

   State  Sex  GeneralHealth  PhysicalHealthDays  MentalHealthDays  \
0      0    0              4                   4                 0   
1      0    1              4                   0                 0   
2      0    1              4                   0                 0   
3      0    0              1                   5                 0   
4      0    0              2                   3                15   

   LastCheckupTime  PhysicalActivities  SleepHours  RemovedTeeth  \
0                3                   1           9             3   
1                3                   1           6             3   
2                3                   0           8             1   
3                3                   1           9             3   
4                3                   1           5             0   

   HadHeartAttack  ...  HeightInMeters  WeightInKilograms    BMI  \
0               0  ...            1.60              71.67  27.99   
1               0  ...            

In [ ]:

# Normalize numerical columns
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Define target variable
target_col = "HadHeartAttack"  # target column
df[target_col] = LabelEncoder().fit_transform(df[target_col])  # Convert "Yes/No" to 1/0

# Split into features and target
X = df.drop(columns=[target_col])
y = df[target_col]

In [ ]:
print(df.head())

   State  Sex  GeneralHealth  PhysicalHealthDays  MentalHealthDays  \
0      0    0              4           -0.053953         -0.513691   
1      0    1              4           -0.508149         -0.513691   
2      0    1              4           -0.508149         -0.513691   
3      0    0              1            0.059596         -0.513691   
4      0    0              2           -0.167502          1.294469   

   LastCheckupTime  PhysicalActivities  SleepHours  RemovedTeeth  \
0                3                   1    1.282643             3   
1                3                   1   -0.708319             3   
2                3                   0    0.618989             1   
3                3                   1    1.282643             3   
4                3                   1   -1.371973             0   

   HadHeartAttack  ...  HeightInMeters  WeightInKilograms       BMI  \
0               0  ...       -1.011468          -0.561179 -0.094374   
1               0  ...      

In [ ]:
import pickle

label_encoder_path = "/content/drive/MyDrive/Deep_Learning/label_encoders.pkl"
scaler_path = "/content/drive/MyDrive/Deep_Learning/scaler.pkl"

with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoders, f)

with open(scaler_path, "wb") as f:
    pickle.dump(scaler, f)


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the neural network, Run the cell to train the model in *epochs
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=16, validation_data=(X_test, y_test))
# Save model weights
model.save("/content/drive/MyDrive/Deep_Learning/health_model.weights.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
483/483 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9212 - loss: 0.3099 - val_accuracy: 0.9379 - val_loss: nan
Epoch 2/3
483/483 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9366 - loss: 0.2248 - val_accuracy: 0.9379 - val_loss: nan
Epoch 3/3
483/483 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9413 - loss: 0.2033 - val_accuracy: 0.9379 - val_loss: nan


In [ ]:
# Evaluate the model
accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9387 - loss: nan
Test Accuracy: 0.9379


In [ ]:
from tensorflow.keras.models import load_model
import os

# Load the saved model
model = load_model("/content/drive/MyDrive/Deep_Learning/health_model.weights.h5")
# Load label encoders
with open("/content/drive/MyDrive/Deep_Learning/label_encoders.pkl", "rb") as f:
    label_encoders = pickle.load(f)

# Load scaler
with open("/content/drive/MyDrive/Deep_Learning/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [ ]:

# Load the original dataset
file_path = "/content/drive/MyDrive/Deep_Learning/heart_2022_no_nans (1).xlsx"
df_original = pd.read_excel(file_path)

# Load encoders and scaler
with open("/content/drive/MyDrive/Deep_Learning/label_encoders.pkl", "rb") as f:
    label_encoders = pickle.load(f)

with open("/content/drive/MyDrive/Deep_Learning/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# Load model
model = load_model("/content/drive/MyDrive/Deep_Learning/health_model.weights.h5")
# Identify columns
categorical_cols = ["State", "Sex", "GeneralHealth", "LastCheckupTime", "PhysicalActivities", "RemovedTeeth"]
numerical_cols = ["PhysicalHealthDays", "MentalHealthDays", "SleepHours"]
target_col = "HadHeartAttack"

# Loop for 5 predictions
for i in range(5):
    random_index = np.random.randint(0, len(df_original))
    random_sample = df_original.iloc[random_index:random_index+1].copy()

    print(f"\nSample {i+1} - Original Data:\n", random_sample)

    for col in categorical_cols:
        le = label_encoders[col]
        try:
            random_sample[col] = le.transform(random_sample[col])
        except ValueError:
            print(f"Unseen label in column '{col}': {random_sample[col].values[0]}")
            random_sample[col] = -1

    random_sample[numerical_cols] = scaler.transform(random_sample[numerical_cols])
    random_sample = random_sample.drop(columns=[target_col])

    prediction = model.predict(random_sample)
    predicted_class = "Yes" if prediction[0][0] > 0.5 else "No"

    print(f"Selected Index: {random_index}")
    print(f"Prediction: HadHeartAttack -> {predicted_class}")
